# Demo for Canopy Height Mapping by GEE
* Open from Google colabo

In [ ]:
!git clone https://github.com/wildflowers315/CH-GEE

In [ ]:
!pip install -r Pycode/requirements.txt

In [ ]:
# Import required libraries
import ee
import geemap
import os
from datetime import datetime

# Import CH-GEE modules
from ch_gee_main import canopy_height_mapper
from l2a_gedi_source import get_gedi_data
from sentinel1_source import get_sentinel1_data
from sentinel2_source import get_sentinel2_data
from for_forest_masking import apply_forest_mask
from random_sampling import create_training_data
from for_upload_download import export_to_asset, export_to_drive, download_to_local

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Define Area of Interest

In [ ]:
Map = geemap.Map()
Map
# Draw a polygon on the map
aoi = Map.draw_last_feature.geometry()

## Set Parameters

In [ ]:
# Set parameters
year = 2022
start_date = '01-01'
end_date = '12-31'
start_date_gedi = '2022-01-01'
end_date_gedi = '2022-12-31'
clouds_th = 0.2
quantile = 'rh100'
model = 'RF'
mask = 'FNF'
gedi_type = 'singleGEDI'

# Random Forest parameters
num_trees_rf = 100
var_split_rf = 3
min_leaf_pop_rf = 1
bag_frac_rf = 0.5
max_nodes_rf = None

# GBM parameters
num_trees_gbm = 100
shr_gbm = 0.1
sampling_rate_gbm = 0.8
max_nodes_gbm = None
loss_gbm = 'squared'

# CART parameters
max_nodes_cart = None
min_leaf_pop_cart = 1

## Run Canopy Height Mapping

In [ ]:
# Run canopy height mapping
classified = canopy_height_mapper(
    aoi=aoi,
    year=year,
    start_date=start_date,
    end_date=end_date,
    start_date_gedi=start_date_gedi,
    end_date_gedi=end_date_gedi,
    clouds_th=clouds_th,
    quantile=quantile,
    model=model,
    mask=mask,
    gedi_type=gedi_type,
    num_trees_rf=num_trees_rf,
    var_split_rf=var_split_rf,
    min_leaf_pop_rf=min_leaf_pop_rf,
    bag_frac_rf=bag_frac_rf,
    max_nodes_rf=max_nodes_rf,
    num_trees_gbm=num_trees_gbm,
    shr_gbm=shr_gbm,
    sampling_rate_gbm=sampling_rate_gbm,
    max_nodes_gbm=max_nodes_gbm,
    loss_gbm=loss_gbm,
    max_nodes_cart=max_nodes_cart,
    min_leaf_pop_cart=min_leaf_pop_cart
)

## Visualize Results

In [ ]:
# Add the classified image to the map
Map.addLayer(classified, {'min': 0, 'max': 50, 'palette': ['white', 'green']}, 'Canopy Height')
Map

## Export Results

In [ ]:
# Export to Earth Engine asset
asset_id = f'users/YOUR_USERNAME/canopy_height_{datetime.now().strftime("%Y%m%d")}'
export_to_asset(classified, 'Canopy Height Map', asset_id)

In [ ]:
# Export to Google Drive
export_to_drive(
    classified,
    'Canopy Height Map',
    'CH-GEE_Outputs',
    f'canopy_height_{datetime.now().strftime("%Y%m%d")}'
)

In [ ]:
# Download to local storage
output_dir = 'outputs'
file_name = f'canopy_height_{datetime.now().strftime("%Y%m%d")}'
download_to_local(classified, output_dir, file_name)